In [1]:
# Import data from Excel sheet
import pandas as pd
df = pd.read_excel('ADNI combined.xlsx', sheet_name='sample')
#print(df)
sid = df['RID']
grp = df['Group at scan date (1=CN, 2=EMCI, 3=LMCI, 4=AD, 5=SMC)']
age = df['Age at scan']
sex = df['Sex (1=female)']
tiv = df['TIV']
field = df['MRI_Field_Strength']
grpbin = (grp > 1) # 1=CN, ...

In [2]:
# Scan for nifti file names
import glob
dataAD = sorted(glob.glob('mwp1_MNI/AD/*.nii.gz'))
dataLMCI = sorted(glob.glob('mwp1_MNI/LMCI/*.nii.gz'))
dataCN = sorted(glob.glob('mwp1_MNI/CN/*.nii.gz'))
dataFiles = dataAD + dataLMCI + dataCN
numfiles = len(dataFiles)
print('Found ', str(numfiles), ' nifti files')

Found  663  nifti files


In [3]:
import re
debug = False
cov_idx = [-1] * numfiles # list; array: np.full((numfiles, 1), -1, dtype=int)
print('Matching covariates for loaded files ...')
for i,id in enumerate(sid):
    p = [j for j,x in enumerate(dataFiles) if re.search('_%04d_' % id, x)] # translate ID numbers to four-digit numbers, get both index and filename
    if len(p)==0:
        if debug: print('Did not find %04d' % id) # did not find Excel sheet subject ID in loaded file selection
    else:
        if debug: print('Found %04d in %s: %s' % (id, p[0], dataFiles[p[0]]))
        cov_idx[p[0]] = i # store Excel index i for data file index p[0]
print('Checking for scans not found in Excel sheet: ', sum(x<0 for x in cov_idx))

labels = pd.DataFrame({'Group':grpbin}).iloc[cov_idx, :]
grps = pd.DataFrame({'Group':grp, 'RID':sid}).iloc[cov_idx, :]

Matching covariates for loaded files ...
Checking for scans not found in Excel sheet:  0


In [4]:
#Load residualized data from disk
import h5py
import numpy as np
hf = h5py.File('residuals_wb_mwp1_MNI.hdf5', 'r')
hf.keys # read keys
labels = np.array(hf.get('labels')) # note: was of data frame type before
images = np.array(hf.get('images'))
hf.close()

In [6]:
import tensorflow as tf
print(tf.__version__)
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
    # device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
set_session(session)

1.15.0


Using TensorFlow backend.


In [5]:
import nibabel as nib

# define FOV to reduce required memory size
x_range_from = 10; x_range_to = 110
y_range_from = 10; y_range_to = 130
z_range_from = 5; z_range_to = 105

data_overlay = sorted(glob.glob('Hippocampus_masks/Hipp*_HarvOx_GMmasked0.5_MNI.nii*'))


In [27]:
# Load CNN model from disk
from keras.models import load_model, Model
#!pip install innvestigate
import innvestigate
import innvestigate.utils as iutils
import numpy as np
from matplotlib import pyplot as plt
import scipy
import csv


# see https://github.com/albermax/innvestigate/blob/master/examples/notebooks/imagenet_compare_methods.ipynb for a list of alternative methods
methods = [ # tuple with method,     params,                  label
#            ("deconvnet",            {},                      "Deconvnet"),
#            ("guided_backprop",      {},                      "Guided Backprop"),
#            ("deep_taylor.bounded",  {"low": -1, "high": 1},  "DeepTaylor"),
#            ("input_t_gradient",     {},                      "Input * Gradient"),
#            ("lrp.z",                {},                      "LRP-Z"),
#            ("lrp.epsilon",          {"epsilon": 1},          "LRP-epsilon"),
            ("lrp.alpha_1_beta_0",   {"neuron_selection_mode":"index"},     "LRP-alpha1beta0"),
]

hipp_nifti = nib.load(data_overlay[0])  # assume it is already 32bit float format
new_data = np.zeros((121, 145, 121), dtype=np.float32) 

for k in [ 7 ]: #range(20):
    mymodel = load_model('newmodel/newmodel_wb_cv%d.hdf5' % (k+1))
    mymodel.name = 'newmodel_wb_cv%d.hdf5' % (k+1)
    #mymodel.summary()
    model_wo_softmax = iutils.keras.graph.model_wo_softmax(mymodel)  ## sometimes raises: ValueError: The name "dense_1" is used 2 times in the model. All layer names should be unique.
    #model_wo_softmax = Model(inputs=mymodel.inputs,
    #                          outputs=iutils.keras.graph.pre_softmax_tensors(mymodel.outputs),
    #                          name=(mymodel.name + '_wo_softmax')) 
    #model_wo_softmax.summary()

    # create analyzer
    analyzers = []
    for method in methods:
        #analyzer = innvestigate.create_analyzer("deep_taylor.bounded", model_wo_softmax, **params )
        analyzer = innvestigate.create_analyzer(method[0], model_wo_softmax, **method[1])
        # Some analyzers require training.
        #   analyzer.fit(test_img, batch_size=30, verbose=1)
        #  analyzers.append(analyzer)
     
    for indx in range(len(grps)):
        test_img = images[indx]
        #test_orig = images_orig[indx]
        #print('test image for subject of binary group: %d' % test_Y[subj_idx, 1]) # first col will indicate CN, second col indicates MCI/AD
        #print('test image for subject of ADNI diagnosis: %d [1-CN, 3-LMCI, 4-AD]' % testgrps.Group.to_numpy(dtype=np.int)[subj_idx])

        ####print('test subject ID %s' % grps.RID.to_numpy(dtype=np.int)[indx])

        test_img = np.reshape(test_img, (1,)+ test_img.shape) # add first subj index again to mimic original array structure
        #test_orig = np.reshape(test_orig, (1,)+ test_orig.shape) # add first subj index again to mimic original array structure

        #for method,analyzer in zip(methods, analyzers):
        a = np.reshape(analyzer.analyze(test_img, neuron_selection=1), test_img.shape[1:4])
        np.clip(a, a_min=0, a_max=None, out=a)
        a = scipy.ndimage.filters.gaussian_filter(a, sigma=0.8) # smooth activity image
        scale = np.quantile(a, 0.99) # no need for abs(a)
        if scale!=0:  # ignore if relevance maps contains only zeros, output will be zero as well
            a = (a/scale)

        a = np.flip(a) # flip all positions
        a = np.transpose(a, (1, 2, 0)) # reorder dimensions from coronal view z*x*y back to x*y*z

        new_data[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to] = a
        nifti = nib.Nifti1Image(new_data, hipp_nifti.affine, hipp_nifti.header)
        print('saving lrp_map_%04d_cv%d.nii' % (grps.RID.iloc[indx], (k+1)))
        nifti.to_filename('lrp_map_%04d_cv%d.nii' % (grps.RID.iloc[indx], (k+1)))

saving lrp_map_0729_cv8.nii
saving lrp_map_1057_cv8.nii
saving lrp_map_5165_cv8.nii
saving lrp_map_5187_cv8.nii
saving lrp_map_0572_cv8.nii
saving lrp_map_5038_cv8.nii
saving lrp_map_5119_cv8.nii
saving lrp_map_1130_cv8.nii
saving lrp_map_0101_cv8.nii
saving lrp_map_5196_cv8.nii
saving lrp_map_1030_cv8.nii
saving lrp_map_5027_cv8.nii
saving lrp_map_5037_cv8.nii
saving lrp_map_5224_cv8.nii
saving lrp_map_5252_cv8.nii
saving lrp_map_6303_cv8.nii
saving lrp_map_5071_cv8.nii
saving lrp_map_0658_cv8.nii
saving lrp_map_0702_cv8.nii
saving lrp_map_1117_cv8.nii
saving lrp_map_1326_cv8.nii
saving lrp_map_5032_cv8.nii
saving lrp_map_5057_cv8.nii
saving lrp_map_5251_cv8.nii
saving lrp_map_5240_cv8.nii
saving lrp_map_4924_cv8.nii
saving lrp_map_0042_cv8.nii
saving lrp_map_0331_cv8.nii
saving lrp_map_0376_cv8.nii
saving lrp_map_0887_cv8.nii
saving lrp_map_5120_cv8.nii
saving lrp_map_5241_cv8.nii
saving lrp_map_5054_cv8.nii
saving lrp_map_0408_cv8.nii
saving lrp_map_0835_cv8.nii
saving lrp_map_4964_

saving lrp_map_4131_cv8.nii
saving lrp_map_4138_cv8.nii
saving lrp_map_4162_cv8.nii
saving lrp_map_4169_cv8.nii
saving lrp_map_4170_cv8.nii
saving lrp_map_4171_cv8.nii
saving lrp_map_4197_cv8.nii
saving lrp_map_4210_cv8.nii
saving lrp_map_4214_cv8.nii
saving lrp_map_4219_cv8.nii
saving lrp_map_4240_cv8.nii
saving lrp_map_4243_cv8.nii
saving lrp_map_4244_cv8.nii
saving lrp_map_4263_cv8.nii
saving lrp_map_4287_cv8.nii
saving lrp_map_4293_cv8.nii
saving lrp_map_4294_cv8.nii
saving lrp_map_4303_cv8.nii
saving lrp_map_4324_cv8.nii
saving lrp_map_4346_cv8.nii
saving lrp_map_4354_cv8.nii
saving lrp_map_4359_cv8.nii
saving lrp_map_4363_cv8.nii
saving lrp_map_4366_cv8.nii
saving lrp_map_4377_cv8.nii
saving lrp_map_4381_cv8.nii
saving lrp_map_4394_cv8.nii
saving lrp_map_4395_cv8.nii
saving lrp_map_4402_cv8.nii
saving lrp_map_4403_cv8.nii
saving lrp_map_4406_cv8.nii
saving lrp_map_4408_cv8.nii
saving lrp_map_4414_cv8.nii
saving lrp_map_4426_cv8.nii
saving lrp_map_4430_cv8.nii
saving lrp_map_4444_

saving lrp_map_4400_cv8.nii
saving lrp_map_4401_cv8.nii
saving lrp_map_4410_cv8.nii
saving lrp_map_4421_cv8.nii
saving lrp_map_4422_cv8.nii
saving lrp_map_4424_cv8.nii
saving lrp_map_4427_cv8.nii
saving lrp_map_4428_cv8.nii
saving lrp_map_4429_cv8.nii
saving lrp_map_4433_cv8.nii
saving lrp_map_4441_cv8.nii
saving lrp_map_4446_cv8.nii
saving lrp_map_4448_cv8.nii
saving lrp_map_4449_cv8.nii
saving lrp_map_4453_cv8.nii
saving lrp_map_4464_cv8.nii
saving lrp_map_4466_cv8.nii
saving lrp_map_4469_cv8.nii
saving lrp_map_4474_cv8.nii
saving lrp_map_4482_cv8.nii
saving lrp_map_4483_cv8.nii
saving lrp_map_4485_cv8.nii
saving lrp_map_4488_cv8.nii
saving lrp_map_4491_cv8.nii
saving lrp_map_4496_cv8.nii
saving lrp_map_4499_cv8.nii
saving lrp_map_4503_cv8.nii
saving lrp_map_4505_cv8.nii
saving lrp_map_4506_cv8.nii
saving lrp_map_4508_cv8.nii
saving lrp_map_4516_cv8.nii
saving lrp_map_4520_cv8.nii
saving lrp_map_4545_cv8.nii
saving lrp_map_4552_cv8.nii
saving lrp_map_4555_cv8.nii
saving lrp_map_4558_